In [4]:
import geopandas as gpd
import xarray as xr
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import gzip
import shutil
import glob

ImportError: DLL load failed while importing _version: The specified procedure could not be found.

In [2]:
# Set data directory
data_dir = "..\..\datasets"

In [3]:
os.listdir(data_dir)

NameError: name 'os' is not defined

In [7]:
aoi = os.path.join(data_dir, 'aoi.gpkg')
gha = gpd.read_file(aoi)

gha.head(5)

## Precipitation pre-processing

### Calculate baseline average

In [ ]:
''''
Study period: 2004 - 2024
'''
# Path to your .tif files
tif_files = sorted(glob.glob("path_to_tif_files/*.tif"))

# Extract dates from filenames (assuming filenames are formatted like chirps-v2.0.1981.07.tif)
dates = [pd.to_datetime(f.split('/')[-1].split('.')[2] + f.split('/')[-1].split('.')[3].replace('.tif', ''), format='%Y.%m') for f in tif_files]

# Load all TIFFs into an xarray DataArray
data_array = xr.concat(
    [xr.open_rasterio(tif).sel(band=1) for tif in tif_files],
    dim=pd.Index(dates, name="time")
)

# Set meaningful coordinates
data_array.name = "precipitation"  # Name the data
data_array.attrs["units"] = "mm"
print(data_array)
